In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import scipy.stats as sps
from sklearn.preprocessing import StandardScaler
from scipy.stats import norm

**Data Cleaning and Variable Creation**

In [0]:
from google.colab import files
uploaded = files.upload()

Saving applications data.csv to applications data (5).csv


In [0]:
import io
data = pd.read_csv(io.BytesIO(uploaded['applications data.csv']))

In [0]:
data.shape

(1000000, 10)

In [0]:
data.head()

,record,date,ssn,firstname,lastname,address,zip5,dob,homephone,fraud_label
0,1,20160101,379070012,XRRAMMTR,SMJETJMJ,6861 EUTST PL,2765,19070626,1797504115,0
1,2,20160101,387482503,MAMSTUJR,RTTEMRRR,7280 URASA PL,57169,19340615,4164239415,1
2,3,20160101,200332444,SZMMUJEZS,EUSEZRAE,5581 RSREX LN,56721,19070626,216537580,0
3,4,20160101,747451317,SJJZSXRSZ,ETJXTXXS,1387 UJZXJ RD,35286,19440430,132144161,0
4,5,20160101,24065868,SSSXUEJMS,SSUUJXUZ,279 EAASA WY,3173,19980315,6101082272,0


In [0]:
data.isnull().sum()

record         0
date           0
ssn            0
firstname      0
lastname       0
address        0
zip5           0
dob            0
homephone      0
fraud_label    0
dtype: int64

Get rid of frivolous data in ssn, address, homephone, dob

In [0]:
data.loc[data['ssn']==999999999,'ssn'] = -data[data['ssn']==999999999]['record']
data.loc[data['address']=='123 MAIN ST','address'] = -data[data['address']=='123 MAIN ST']['record']
data.loc[data['homephone']==9999999999,'homephone'] = -data[data['homephone']==9999999999]['record']
data.loc[data['dob']==19070626,'dob'] = -data[data['dob']==19070626]['record']

Change "date" to datetime and all other fields to string

In [0]:
data.dtypes

record          int64
date            int64
ssn             int64
firstname      object
lastname       object
address        object
zip5            int64
dob             int64
homephone       int64
fraud_label     int64
dtype: object

In [0]:
data['appyear'] = (data['date'] / 10000).astype('int')
data['appmonth'] = ((data['date'] - data['appyear'] * 10000) / 100).astype('int')
data['appday'] = data['date'] - data['appyear'] * 10000 - data['appmonth'] * 100
data['date'] = pd.to_datetime(data.appyear*10000+data.appmonth*100+data.appday,format='%Y%m%d')
data = data.drop(columns = ['appyear','appmonth','appday'])

In [0]:
data['ssn'] = data['ssn'].astype(str)
data['firstname'] = data['firstname'].astype(str)
data['lastname'] = data['lastname'].astype(str)
data['address'] = data['address'].astype(str)
data['zip5'] = data['zip5'].astype(str)
data['dob'] = data['dob'].astype(str)
data['homephone'] = data['homephone'].astype(str)

In [0]:
data.head()

,record,date,ssn,firstname,lastname,address,zip5,dob,homephone,fraud_label
0,1,2016-01-01,379070012,XRRAMMTR,SMJETJMJ,6861 EUTST PL,2765,-1,1797504115,0
1,2,2016-01-01,387482503,MAMSTUJR,RTTEMRRR,7280 URASA PL,57169,19340615,4164239415,1
2,3,2016-01-01,200332444,SZMMUJEZS,EUSEZRAE,5581 RSREX LN,56721,-3,216537580,0
3,4,2016-01-01,747451317,SJJZSXRSZ,ETJXTXXS,1387 UJZXJ RD,35286,19440430,132144161,0
4,5,2016-01-01,24065868,SSSXUEJMS,SSUUJXUZ,279 EAASA WY,3173,19980315,6101082272,0


In [0]:
data.dtypes

record                  int64
date           datetime64[ns]
ssn                    object
firstname              object
lastname               object
address                object
zip5                   object
dob                    object
homephone              object
fraud_label             int64
dtype: object

Create combination fields

In [0]:
data['name'] = data['firstname']+data['lastname']
data['fulladdress'] = data['address']+data['zip5']

data['name-dob'] = data['name']+data['dob']
data['name-fulladdress'] = data['name']+data['fulladdress']
data['name-homephone'] = data['name']+data['homephone']
data['dob-fulladdress'] = data['dob']+data['fulladdress']
data['dob-homephone'] = data['dob']+data['homephone']
data['fulladdress-homephone'] = data['fulladdress']+data['homephone']

data['name-dob-fulladdress'] = data['name']+data['dob']+data['fulladdress']
data['name-dob-homephone'] = data['name']+data['dob']+data['homephone']
data['name-fulladdress-homephone'] = data['name']+data['fulladdress']+data['homephone']
data['dob-fulladdress-homephone'] = data['dob']+data['fulladdress']+data['homephone']

data['name-dob-fulladdress-homephone'] = data['name']+data['dob']+data['fulladdress']+data['homephone']

for field in ['firstname','lastname','address','zip5','dob','homephone','name','fulladdress','name-dob']:
    data['ssn-{}'.format(field)] = data['ssn']+data[field]

In [0]:
attributes = list(data.columns)
attributes.remove('fraud_label')
attributes.remove('date')
attributes.remove('firstname')
attributes.remove('lastname')
attributes.remove('zip5')
attributes.remove('record')
print(len(attributes))

26


In [0]:
attributes

['ssn',
 'address',
 'dob',
 'homephone',
 'name',
 'fulladdress',
 'name-dob',
 'name-fulladdress',
 'name-homephone',
 'dob-fulladdress',
 'dob-homephone',
 'fulladdress-homephone',
 'name-dob-fulladdress',
 'name-dob-homephone',
 'name-fulladdress-homephone',
 'dob-fulladdress-homephone',
 'name-dob-fulladdress-homephone',
 'ssn-firstname',
 'ssn-lastname',
 'ssn-address',
 'ssn-zip5',
 'ssn-dob',
 'ssn-homephone',
 'ssn-name',
 'ssn-fulladdress',
 'ssn-name-dob']

Build candidate variables

In [0]:
finalDF = data[['record','date']]

for entity in attributes:
  data1 = data[['record','date',entity]].copy()
  data2 = data[['record','date',entity]].copy()
  days = [0,1,3,7,14,30]

  for day in days:
    temp_name = '{}_day'.format(day)
    data2[temp_name] = data2['date']+dt.timedelta(day)
    
  data3 = data1.merge(data2, on=entity)
    
# Day since the field value was last seen
  temp = data3[(data3['record_x']>data3['record_y'])][['record_x','date_y']].groupby('record_x').last()['date_y'].reset_index()
  temp.columns = ['record','last_seen']
  data1 = data1.merge(temp, how='left', on='record')
  data1['#_of_days_since'] = data1['date'] - data1['last_seen']
  data1['#_of_days_since'] = data1['#_of_days_since'].dt.days
  data1['#_of_days_since_start'] = (data1['date']-data1['date'].min()).dt.days
  data1['#_of_days_since'] = data1['#_of_days_since'].fillna(data1['#_of_days_since_start'])
  data1['#_of_days_since'] = data1['#_of_days_since'].apply(lambda x:min(x,60))

  data1.drop(columns=['#_of_days_since_start','last_seen'], inplace=True)

    
# Velocity variables
  for day in days:
    temp = data3[(data3['record_x']>=data3['record_y'])&(data3['date_x']<=data3['{}_day'.format(day)])]
    tempCount = temp[['record_x','record_y']].groupby('record_x').count().reset_index()
    tempCount.columns = ['record','{}_day_count'.format(day)]
    data1 = data1.merge(tempCount, left_on='record', right_on='record')

# Relative velocity variables 
  for i in [3,7,14,30]:
    temp_name = '1_day-{}_day_avg'.format(i)
    data1[temp_name] = data1['1_day_count']/(data1['{}_day_count'.format(i)]/i)
  
  data1.drop(columns=['date',entity], inplace=True)
  data1 = data1.set_index('record')
  data1 = data1.add_prefix('{}_'.format(entity))
  finalDF = finalDF.merge(data1, left_on='record',right_index=True)

In [0]:
# Cross entity variables 
for entity in ['ssn','dob','homephone','name','fulladdress']:
  entity_list = ['ssn','dob','homephone','name','fulladdress']
  entity_list.remove(entity)
  for entity1 in entity_list:
    data4 = data[['record','date',entity,entity1]].copy()
    data5 = data[['record','date',entity,entity1]].copy()
    days = [0,1,3,7,14,30]
    
    for day in days:
      temp_name = '{}_day'.format(day)
      data5[temp_name] = data5['date']+dt.timedelta(day)
    
    data6 = data4.merge(data5, on=entity)
    
    for day in days:
      temp = data6[(data6['record_x']>=data6['record_y'])&(data6['date_x']<=data6['{}_day'.format(day)])]
      tempCount = temp[['record_x','{}_y'.format(entity1)]].groupby(['record_x'])['{}_y'.format(entity1)].nunique().reset_index()
      tempCount.columns = ['record','{}_day_cross_count'.format(day)]
      data4 = data4.merge(tempCount, left_on='record',right_on='record')

    data4.drop(columns=['date',entity,entity1],inplace=True) 
    data4 = data4.set_index('record')
    data4 = data4.add_prefix('{}_{}_'.format(entity,entity1))
    finalDF = finalDF.merge(data4, left_on='record', right_index=True)

Create a risk table: likelyhood of fraud for that day of the week

In [0]:
finalDF['weekday'] = finalDF['date'].dt.dayofweek
finalDF = finalDF.merge(data[['record','fraud_label']])
tt = finalDF[finalDF.date<'2016-11-01'].copy()
oot = finalDF[finalDF.date>='2016-11-01'].copy()
c = 4
nmid = 20
y_avg = tt['fraud_label'].mean()
y_weekday = tt.groupby('weekday')['fraud_label'].mean()
num_instances_weekday = tt.groupby('weekday').size()
y_weekday_smooth = y_avg + (y_weekday - y_avg) / (1 + np.exp(-(num_instances_weekday-nmid)/c))
finalDF['weekday_risk'] = finalDF['weekday'].map(y_weekday_smooth)

In [0]:
for i in range(7):
  print(finalDF[finalDF.weekday==i]['weekday_risk'].unique())

[0.01347979]
[0.01407035]
[0.01516916]
[0.01498115]
[0.01449888]
[0.01496822]
[0.01367351]


In [0]:
finalDF.shape

(1000000, 411)

In [0]:
finalDF.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
record,1000000.0,500000.500000,288675.278932,1.00000,250000.750000,500000.500000,750000.250000,1000000.000000
ssn_#_of_days_since,1000000.0,53.310300,15.191059,0.00000,60.000000,60.000000,60.000000,60.000000
ssn_0_day_count,1000000.0,1.007305,0.223356,1.00000,1.000000,1.000000,1.000000,21.000000
ssn_1_day_count,1000000.0,1.014924,0.381210,1.00000,1.000000,1.000000,1.000000,34.000000
ssn_3_day_count,1000000.0,1.020140,0.423193,1.00000,1.000000,1.000000,1.000000,34.000000
...,...,...,...,...,...,...,...,...
fulladdress_name_14_day_cross_count,1000000.0,1.033946,0.594816,1.00000,1.000000,1.000000,1.000000,30.000000
fulladdress_name_30_day_cross_count,1000000.0,1.035916,0.607661,1.00000,1.000000,1.000000,1.000000,30.000000
weekday,1000000.0,3.014974,1.994270,0.00000,1.000000,3.000000,5.000000,6.000000
fraud_label,1000000.0,0.014393,0.119104,0.00000,0.000000,0.000000,0.000000,1.000000


In [0]:
from google.colab import files
finalDF.describe().transpose().to_csv('variable.csv') 

In [0]:
from google.colab import files
finalDF.to_csv('finalDF.csv')